### Imports

In [45]:
from bs4 import BeautifulSoup
import requests
import json
import re
import string
from pymongo import MongoClient
from nltk.corpus import stopwords
import pyarabic.araby as araby
from pyarabic.number import ArNumbers
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer as lem
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer

### Loading the BeautifulSoup object

In [46]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

### Retrieving the divs that contain the links to articles

In [47]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    # links
    link = article.find(name='a').get('href')
    # article titles
    title = article.find(name='a').get('title')
    articals.append({title : link})

### Storing links and titles in a json file

In [48]:
with open('data/hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)

### Retrieving the divs that contain the links to articles

In [49]:
with open('data/hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

### Storing articles and their respective titles in a json file

In [50]:
with open('data/hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})
    json.dump(contents, f, ensure_ascii=False, indent=2)

### Uploading the two json files in a MongoDB database

In [51]:
with open('data/hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('data/hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

### Loading the first article for the NLP pipeline

In [52]:
text = art_content[0]['content']
text.encode('utf-8')
print("Raw text: ",text)

Raw text:  ما زالت تصريحات عبد اللطيف الجواهري، والي بنك المغرب، عقب آخر اجتماع للمجلس الإداري للبنك المركزي، موضوع تحليل بين أوساط الخبراء الاقتصاديين بالمغرب والخارج، حيث لم يجزم بشأن المحافظة على معدل الفائدة الرئيسي الحالي عند 3 في المائة من عدمه، خلال الاجتماع المقبل للمجلس بحلول يونيو المقبل، مكتفيا بالتأكيد على أن القرار سيكون رهينا بالمؤشرات المسجلة خلال ثاني فصول السنة الجارية؛ ما يترك المجال مفتوحا لجميع التوقعات.هل الحكومة مستعدة لمواجهة موجة تضخمية جديدة بعد رمضان؟. سؤال ما فتئ الخبراء الاقتصاديون يطرحونه في ظل غموض التوقعات المرتبطة بالتضخم وتوجه بنك المغرب، خصوصا خلال الفترتين الماضية والحالية، حيث تعيش المملكة فترة ركود تضخمي؛ إلا أن ارتفاع أسعار الخضر وبعض المواد الاستهلاكية بشكل تدريجي منذ النصف الثاني من الشهر الفضيل وتجدد الحديث عن رفع الدعم عن غاز البوتان عززا التوقعات بشأن موجة غلاء مرتقبة.بذلت الحكومة، منذ نهاية السنة الماضية، مجهودات مهمة من أجل ضمان توفير منتوجات بأسعار منخفضة نسبيا على موائد المغاربة خلال رمضان، فكثفت حملات مراقبة الأسعار ومكافحة الاحتكار، واست

### Tokenization

In [53]:
tokens = araby.tokenize(text)
sent_tokens = araby.sentence_tokenize(text)
print("Word tokenized list: ", tokens)
print("Sentence tokenized list: ", sent_tokens)
print("Word tokenized list length: ", len(tokens))
print("Sentence tokenized list length: ", len(sent_tokens))

Word tokenized list:  ['ما', 'زالت', 'تصريحات', 'عبد', 'اللطيف', 'الجواهري', '،', 'والي', 'بنك', 'المغرب', '،', 'عقب', 'آخر', 'اجتماع', 'للمجلس', 'الإداري', 'للبنك', 'المركزي', '،', 'موضوع', 'تحليل', 'بين', 'أوساط', 'الخبراء', 'الاقتصاديين', 'بالمغرب', 'والخارج', '،', 'حيث', 'لم', 'يجزم', 'بشأن', 'المحافظة', 'على', 'معدل', 'الفائدة', 'الرئيسي', 'الحالي', 'عند', '3', 'في', 'المائة', 'من', 'عدمه', '،', 'خلال', 'الاجتماع', 'المقبل', 'للمجلس', 'بحلول', 'يونيو', 'المقبل', '،', 'مكتفيا', 'بالتأكيد', 'على', 'أن', 'القرار', 'سيكون', 'رهينا', 'بالمؤشرات', 'المسجلة', 'خلال', 'ثاني', 'فصول', 'السنة', 'الجارية', '؛', 'ما', 'يترك', 'المجال', 'مفتوحا', 'لجميع', 'التوقعات', '.', 'هل', 'الحكومة', 'مستعدة', 'لمواجهة', 'موجة', 'تضخمية', 'جديدة', 'بعد', 'رمضان', '؟.', 'سؤال', 'ما', 'فتئ', 'الخبراء', 'الاقتصاديون', 'يطرحونه', 'في', 'ظل', 'غموض', 'التوقعات', 'المرتبطة', 'بالتضخم', 'وتوجه', 'بنك', 'المغرب', '،', 'خصوصا', 'خلال', 'الفترتين', 'الماضية', 'والحالية', '،', 'حيث', 'تعيش', 'المملكة', 'فترة', 'ركود

### Removing punctuation

In [54]:
ar_punct = ''')(+`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”،.”…“–ـ”.'''
en_punct = string.punctuation
punct_lst = ar_punct + en_punct

In [55]:
tokens = [token for token in tokens if token not in punct_lst]
sent_tokens = [sent.translate(str.maketrans('', '', punct_lst)) for sent in sent_tokens]
print("Tokenized text without punctuation: ",tokens)
print("Sentence tokenized text without punctuation: ",sent_tokens)
print("Length of tokenized text without punctuation: ", len(tokens))
print("Length of sentence tokenized text without punctuation: ", len(sent_tokens))

Tokenized text without punctuation:  ['ما', 'زالت', 'تصريحات', 'عبد', 'اللطيف', 'الجواهري', 'والي', 'بنك', 'المغرب', 'عقب', 'آخر', 'اجتماع', 'للمجلس', 'الإداري', 'للبنك', 'المركزي', 'موضوع', 'تحليل', 'بين', 'أوساط', 'الخبراء', 'الاقتصاديين', 'بالمغرب', 'والخارج', 'حيث', 'لم', 'يجزم', 'بشأن', 'المحافظة', 'على', 'معدل', 'الفائدة', 'الرئيسي', 'الحالي', 'عند', '3', 'في', 'المائة', 'من', 'عدمه', 'خلال', 'الاجتماع', 'المقبل', 'للمجلس', 'بحلول', 'يونيو', 'المقبل', 'مكتفيا', 'بالتأكيد', 'على', 'أن', 'القرار', 'سيكون', 'رهينا', 'بالمؤشرات', 'المسجلة', 'خلال', 'ثاني', 'فصول', 'السنة', 'الجارية', 'ما', 'يترك', 'المجال', 'مفتوحا', 'لجميع', 'التوقعات', 'هل', 'الحكومة', 'مستعدة', 'لمواجهة', 'موجة', 'تضخمية', 'جديدة', 'بعد', 'رمضان', 'سؤال', 'ما', 'فتئ', 'الخبراء', 'الاقتصاديون', 'يطرحونه', 'في', 'ظل', 'غموض', 'التوقعات', 'المرتبطة', 'بالتضخم', 'وتوجه', 'بنك', 'المغرب', 'خصوصا', 'خلال', 'الفترتين', 'الماضية', 'والحالية', 'حيث', 'تعيش', 'المملكة', 'فترة', 'ركود', 'تضخمي', 'إلا', 'أن', 'ارتفاع', 'أسعار

### Removing stopwords

In [56]:
stop_words = set(stopwords.words('arabic'))
print("Stop words: ",stop_words)
tokens = [word for word in tokens if word not in stop_words]
sent_tokens = [''.join([x for x in re.split(r'(\W+)', sent) if x not in stop_words]) for sent in sent_tokens]

print("Tokenized text without stopwords: ",tokens)
print("Sentence tokenized text without stopwords: ",sent_tokens)
print("Length of tokenized text without stopwords: ", len(tokens))
print("Length of sentence tokenized text without stopwords: ", len(sent_tokens))

Stop words:  {'اللتين', 'ف', 'هَاتِه', 'شمال', 'كذلك', 'لا سيما', 'إنا', 'بهما', 'فلس', 'لبيك', 'ستون', 'وإن', 'ممن', 'آهٍ', 'مكانكم', 'إياهم', 'أول', 'عشرين', 'حجا', 'تِي', 'غداة', 'سبعمئة', 'لستم', 'رأى', 'هو', 'مايو', 'ذلك', 'انقلب', 'والذي', 'تشرين', 'شيكل', 'آهاً', 'حبيب', 'هيت', 'عدا', 'نا', 'ثمة', 'أربعاء', 'خلافا', 'عليك', 'لكم', 'منها', 'هن', 'حبذا', 'إلَيْكَ', 'تلكم', 'سوى', 'رُبَّ', 'اللواتي', 'لسن', 'تعلَّم', 'تفعلان', 'أنا', 'كليهما', 'كرب', 'هبّ', 'لم', 'أُفٍّ', 'فلا', 'حيَّ', 'بيد', 'أن', 'ك', 'إي', 'فيه', 'كان', 'فيها', 'ز', 'فوق', 'ثمّ', 'وا', 'أحد', 'ثلاثمئة', 'وهب', 'بخ', 'لن', 'ص', 'ذا', 'ءَ', 'هيا', 'هللة', 'إذن', 'ثلاثين', 'هاتان', 'أعطى', 'خامس', 'هذا', 'يا', 'خبَّر', 'آ', 'تعسا', 'جمعة', 'إيانا', 'بات', 'هاهنا', 'إليكَ', 'ذيت', 'إزاء', 'أمد', 'كما', 'ميم', 'بسّ', 'كسا', 'سوف', 'كأين', 'أطعم', 'أبٌ', 'دون', 'خميس', 'نيف', 'كاد', 'مئة', 'لو', 'نيسان', 'كى', 'أصلا', 'اربعين', 'أى', 'ؤ', 'سبعة', 'ته', 'سبعون', 'م', 'قاف', 'ج', 'أمّا', 'زعم', 'ألا', 'لات', 'ثان', 'سر

### Convert numbers to words

In [57]:
arnum = ArNumbers()
num_tokens = []
for i,token in enumerate(tokens):
    if tokens[i].isdigit():
        tokens[i] = arnum.int2str(tokens[i])
        num_tokens.append(tokens[i])
print("Number to string: ", tokens)
print("Numerical tokens: ", num_tokens)

Number to string:  ['زالت', 'تصريحات', 'عبد', 'اللطيف', 'الجواهري', 'والي', 'بنك', 'المغرب', 'عقب', 'آخر', 'اجتماع', 'للمجلس', 'الإداري', 'للبنك', 'المركزي', 'موضوع', 'تحليل', 'أوساط', 'الخبراء', 'الاقتصاديين', 'بالمغرب', 'والخارج', 'يجزم', 'بشأن', 'المحافظة', 'معدل', 'الفائدة', 'الرئيسي', 'الحالي', 'ثلاث', 'المائة', 'عدمه', 'خلال', 'الاجتماع', 'المقبل', 'للمجلس', 'بحلول', 'المقبل', 'مكتفيا', 'بالتأكيد', 'القرار', 'سيكون', 'رهينا', 'بالمؤشرات', 'المسجلة', 'خلال', 'فصول', 'السنة', 'الجارية', 'يترك', 'المجال', 'مفتوحا', 'لجميع', 'التوقعات', 'الحكومة', 'مستعدة', 'لمواجهة', 'موجة', 'تضخمية', 'جديدة', 'رمضان', 'سؤال', 'فتئ', 'الخبراء', 'الاقتصاديون', 'يطرحونه', 'ظل', 'غموض', 'التوقعات', 'المرتبطة', 'بالتضخم', 'وتوجه', 'بنك', 'المغرب', 'خصوصا', 'خلال', 'الفترتين', 'الماضية', 'والحالية', 'تعيش', 'المملكة', 'فترة', 'ركود', 'تضخمي', 'ارتفاع', 'أسعار', 'الخضر', 'وبعض', 'المواد', 'الاستهلاكية', 'بشكل', 'تدريجي', 'النصف', 'الثاني', 'الشهر', 'الفضيل', 'وتجدد', 'الحديث', 'رفع', 'الدعم', 'غاز', 'البو

### Normalization

In [58]:
ArListem = ArabicLightStemmer()
# removing numerical tokens
for i in num_tokens:
    tokens.remove(i)
tokens = [ArListem.normalize(token) for token in tokens]
print("Normalized token list: ", tokens)

Normalized token list:  ['زالت', 'تصريحات', 'عبد', 'اللطيف', 'الجواهري', 'والي', 'بنك', 'المغرب', 'عقب', 'اخر', 'اجتماع', 'للمجلس', 'الاداري', 'للبنك', 'المركزي', 'موضوع', 'تحليل', 'اوساط', 'الخبراء', 'الاقتصاديين', 'بالمغرب', 'والخارج', 'يجزم', 'بشان', 'المحافظه', 'معدل', 'الفاءده', 'الرءيسي', 'الحالي', 'الماءه', 'عدمه', 'خلال', 'الاجتماع', 'المقبل', 'للمجلس', 'بحلول', 'المقبل', 'مكتفيا', 'بالتاكيد', 'القرار', 'سيكون', 'رهينا', 'بالمءشرات', 'المسجله', 'خلال', 'فصول', 'السنه', 'الجاريه', 'يترك', 'المجال', 'مفتوحا', 'لجميع', 'التوقعات', 'الحكومه', 'مستعده', 'لمواجهه', 'موجه', 'تضخميه', 'جديده', 'رمضان', 'سءال', 'فتء', 'الخبراء', 'الاقتصاديون', 'يطرحونه', 'ظل', 'غموض', 'التوقعات', 'المرتبطه', 'بالتضخم', 'وتوجه', 'بنك', 'المغرب', 'خصوصا', 'خلال', 'الفترتين', 'الماضيه', 'والحاليه', 'تعيش', 'المملكه', 'فتره', 'ركود', 'تضخمي', 'ارتفاع', 'اسعار', 'الخضر', 'وبعض', 'المواد', 'الاستهلاكيه', 'بشكل', 'تدريجي', 'النصف', 'الثاني', 'الشهر', 'الفضيل', 'وتجدد', 'الحديث', 'رفع', 'الدعم', 'غاز', 'البوتان

### Stemming

In [59]:
stemmed_tokens = [ArListem.light_stem(token) for token in tokens]
print("Stemmed token list: ", stemmed_tokens)

Stemmed token list:  ['زال', 'تصريح', 'عبد', 'لطيف', 'جواهر', 'لي', 'نك', 'مغرب', 'عقب', 'خر', 'جتماع', 'مجلس', 'ادار', 'بنك', 'مركز', 'موضوع', 'حليل', 'وساط', 'خبراء', 'اقتصادي', 'مغرب', 'خارج', 'جزم', 'شان', 'محافظه', 'معدل', 'فاءده', 'رءيس', 'حال', 'ماءه', 'عدم', 'خلال', 'اجتماع', 'مقبل', 'مجلس', 'حلول', 'مقبل', 'مكتفي', 'تاكيد', 'قرار', 'كو', 'ره', 'مءشر', 'مسجله', 'خلال', 'صول', 'سنه', 'جاريه', 'تر', 'مجال', 'مفتوح', 'جميع', 'توقع', 'حكومه', 'مستعد', 'مواجه', 'موج', 'ضخم', 'جديد', 'رمض', 'سءال', 'تء', 'خبراء', 'اقتصادي', 'طرح', 'ظل', 'غموض', 'توقع', 'مرتبطه', 'تضخم', 'وج', 'نك', 'مغرب', 'خصوص', 'خلال', 'فتر', 'ماضيه', 'حاليه', 'عيش', 'مملكه', 'ره', 'ركود', 'ضخم', 'رتفاع', 'سعار', 'خضر', 'بعض', 'مواد', 'الاستهلاكيه', 'شكل', 'دريج', 'نصف', 'ثان', 'شهر', 'فضيل', 'جدد', 'حديث', 'رفع', 'دعم', 'غاز', 'بو', 'عزز', 'توقع', 'شان', 'موج', 'غلاء', 'مرتقب', 'ذلت', 'حكومه', 'هاي', 'سنه', 'ماضيه', 'مجهود', 'مهم', 'ضم', 'وفير', 'منتوج', 'اسعار', 'منخفض', 'سبيا', 'مواءد', 'مغاربه', 'خلال', 'رمض',

### Lemmatization

In [60]:
lemmer = lem.Lemmatizer()
lemmatized_tokens = [lemmer.lemmatize(token) for token in tokens]
print("Lemmatized token list: ", lemmatized_tokens)

Lemmatized token list:  ['زالت', 'تصريح', 'عبد', 'لطيف', 'جواهر', 'والى', 'بن', 'مغرب', 'عقب', 'اخر', 'اجتماع', 'مجلس', 'الاداري', 'لبن', 'مركز', 'موضوع', 'تحليل', 'اوساط', 'خبراء', 'اقتصادي', 'مغرب', 'خارج', 'جزم', 'بش', 'المحافظه', 'معدل', 'الفاءده', 'الرءيسي', 'حال', 'الماءه', 'عدم', 'خلال', 'اجتماع', 'مقبل', 'مجلس', 'حلول', 'مقبل', 'مكتف', 'بالتاكيد', 'قرار', 'كون', 'رهين', 'بالمءشرات', 'المسجله', 'خلال', 'فصول', 'لسن', 'الجاريه', 'ترك', 'مجال', 'مفتوح', 'جميع', 'توقع', 'الحكومه', 'مستعد', 'مواجه', 'موجه', 'تضخم', 'جديد', 'رمض', 'سءال', 'فتء', 'خبراء', 'اقتصادي', 'طرح', 'ظل', 'غموض', 'توقع', 'المرتبطه', 'تضخم', 'توجه', 'بن', 'مغرب', 'خصوص', 'خلال', 'فترة', 'الماضيه', 'والحاليه', 'تعيش', 'المملكه', 'فتر', 'ركود', 'تضخم', 'ارتفاع', 'اسعار', 'خضر', 'بعض', 'مواد', 'الاستهلاكيه', 'شكل', 'تدريج', 'نصف', 'ثاني', 'شهر', 'فضيل', 'تجدد', 'حديث', 'رفع', 'دعم', 'غاز', 'بوتان', 'عزز', 'توقع', 'بش', 'موجه', 'غلاء', 'مرتقب', 'بذل', 'الحكومه', 'نهايه', 'لسن', 'الماضيه', 'مجهود', 'مهم', 'ضمان', 'تو

### Stemming vs lemmatization comparison

In [61]:
print("Original token list: ", tokens)
print("Stemmed token list: ", stemmed_tokens)
print("Lemmatized token list: ", lemmatized_tokens)

Original token list:  ['زالت', 'تصريحات', 'عبد', 'اللطيف', 'الجواهري', 'والي', 'بنك', 'المغرب', 'عقب', 'اخر', 'اجتماع', 'للمجلس', 'الاداري', 'للبنك', 'المركزي', 'موضوع', 'تحليل', 'اوساط', 'الخبراء', 'الاقتصاديين', 'بالمغرب', 'والخارج', 'يجزم', 'بشان', 'المحافظه', 'معدل', 'الفاءده', 'الرءيسي', 'الحالي', 'الماءه', 'عدمه', 'خلال', 'الاجتماع', 'المقبل', 'للمجلس', 'بحلول', 'المقبل', 'مكتفيا', 'بالتاكيد', 'القرار', 'سيكون', 'رهينا', 'بالمءشرات', 'المسجله', 'خلال', 'فصول', 'السنه', 'الجاريه', 'يترك', 'المجال', 'مفتوحا', 'لجميع', 'التوقعات', 'الحكومه', 'مستعده', 'لمواجهه', 'موجه', 'تضخميه', 'جديده', 'رمضان', 'سءال', 'فتء', 'الخبراء', 'الاقتصاديون', 'يطرحونه', 'ظل', 'غموض', 'التوقعات', 'المرتبطه', 'بالتضخم', 'وتوجه', 'بنك', 'المغرب', 'خصوصا', 'خلال', 'الفترتين', 'الماضيه', 'والحاليه', 'تعيش', 'المملكه', 'فتره', 'ركود', 'تضخمي', 'ارتفاع', 'اسعار', 'الخضر', 'وبعض', 'المواد', 'الاستهلاكيه', 'بشكل', 'تدريجي', 'النصف', 'الثاني', 'الشهر', 'الفضيل', 'وتجدد', 'الحديث', 'رفع', 'الدعم', 'غاز', 'البوتان',

### Machine learning approach for PoS tagging

In [62]:
pos_tagger = FarasaPOSTagger()
tagged_text =  pos_tagger.tag(text) # [pos_tagger.tag(token) for token in lemmatized_tokens]
print("POS Tagged",tagged_text)

POS Tagged S/S ما/PART زال +ت/V+PRON تصريح +ات/NOUN+NSUFF-FP عبد/NOUN-MS ال+ لطيف/DET+NOUN-MS ال+ جواهري/DET+NOUN-MS ،/PUNC والي/NOUN-MS بنك/NOUN-MS ال+ مغرب/DET+NOUN-MS ،/PUNC عقب/NOUN-MS آخر/NOUN-MS اجتماع/NOUN-MS ل+/PREP ال+ مجلس/DET+NOUN-MS ال+ إداري/DET+ADJ-MS ل+/PREP ال+ بنك/DET+NOUN-MS ال+ مركزي/DET+ADJ-MS ،/PUNC موضوع/NOUN-MS تحليل/NOUN-MS بين/NOUN-MS أوساط/NOUN-FP ال+ خبراء/DET+NOUN-MP ال+ اقتصادي +ين/DET+NOUN+NSUFF-MP ب+/PREP ال+ مغرب/DET+NOUN-MS و+/CONJ ال+ خارج/DET+NOUN-MS ،/PUNC حيث/PART لم/PART يجزم/V ب+/PREP شأن/NOUN-MS ال+ محافظ +ة/DET+NOUN+NSUFF-FS على/PREP معدل/NOUN-MS ال+ فائد +ة/DET+NOUN+NSUFF-FS ال+ رئيسي/DET+ADJ-MS ال+ حالي/DET+ADJ-MS عند/NOUN-MS 3/NUM-MP في/PREP ال+ مائ +ة/DET+NUM+NSUFF-FS من/PREP عدم/NOUN-MS +ه/PRON ،/PUNC خلال/NOUN-MS ال+ اجتماع/DET+NOUN-MS ال+ مقبل/DET+ADJ-MS ل+/PREP ال+ مجلس/DET+NOUN-MS ب+/PREP حلول/NOUN-FP يونيو/NOUN-MS ال+ مقبل/DET+ADJ-MS ،/PUNC مكتفي/ADJ-MS +ا/CASE ب+/PREP ال+ تأكيد/DET+NOUN-MS على/PREP أن/PART ال+ قرار/DET+NOUN-MS س+/FUT_

In [63]:
# free up memory
del pos_tagger

### Rule-Based approach for PoS tagging

In [64]:
def tag(token):
    rules_1_2 = [r'^\s*\u0643\u0627\u0644.*\s*$', r'^\s*\u0628\u0627\u0644.*\s*$', r'^\s*\u0641\u0627\u0644.*\s*$', 
                r'^\s*\u0648\u0627\u0644.*\s*$',r'^\s*\w*\u0627\u0626\u0647\b', r'^\s*\w*\u0627\u0626\u0643\b',
                r'^\s*\w*\u0627\u0626\u064a\b', r'^\s*\w*\u0627\u0624\u0643\b', r'^\s*\w*\u0627\u0624\u0647\b',
                r'^\s*\w*\u0627\u0621\u0643\b',r'^\s*\w*\u0627\u0621\u0647\b',r'^\s*\w*\u0647\u0645\u0627\b',
                r'^\s*\w*\u0643\u0645\u0627\b']
    rules_3_4 = [r'^\s*\u0633\u064a.*\s*$', r'^\s*\u0633\u062a.*\s*$', r'^\s*\u0633\u0646.*\s*$', r'^\s*\u0633\u0623.*\s*$',
                r'^\s*\u0633\u0627.*\s*$',r'^\s*\u0644\u0627.*\s*$',r'^\s*\u0644\u0623.*\s*$',r'^\s*\u0644\u0646.*\s*$',
                r'^\s*\u0644\u062a.*\s*$',r'^\s*\u0644\u064a.*\s*$', r'^\s*\w*\u064a\b',r'^\s*\w*\u0647\b',r'^\s*\w*\u0643\b',
                r'^\s*\w*\u0627\b',r'^\s*\w*\u0646\b',r'^\s*\w*\u0648\b']
    rules_5_6 = [r'^\s*\w{3}.*\u0649\s*$', r'^\s*\w{2}\u0648\w{1}\s*$',r'^\s*\w{2}\u0627\u0621\s*$',r'^\s*\w*\u0627\u062a\s*$']
    rule_7 = r'^\s*(\u064a|\u0646)\w*(\u0648\u0646|\u064a\u0646)\s*$'
    rules_8_9 = [r'^\s*[^(\u064a|\u0646)]\w*(\u0648\u0646|\u064a\u0646)\s*$',r'^\s*\u0645\w{1}\u0627\w{2}\s*$',
                 r'^\s*\u0645\w{2}\u064a\w{1}\s*$', r'^\s*\u0645\w{2}\u0627\w{1}\s*$', r'^\s*\u0645\w{1}\u062a\w{2}\s*$',
                 r'^\s*\u0645\u0646\w{3}\s*$',r'^\s*\u0645\w{2}\u0648\w{1}\s*$',r'^\s*\u0645\u062a\w{3}\s*$',
                 r'^\s*\u0645\w{2}(.)\1\s*$']
    rule_10 = [r'^\s*\w{2}\u0627\w{1}\u064a\w{1}\s*$',r'^s*\u0623\w{1}\u0627\w{1}\u064a\w{1}\s*$',
                 r'^\s*\u0645\w{2}\u0627\w{1}\u064a\w{1}\s*$',r'^\s*\w{1}\u0648\u0627\w{1}\u064a\w{1}\s*$']
    rule_11 = [r'^\s*\u0627\u0633\u062a\w{3}\s*$',r'^\s*\u0627\w{1}(.)\u0648\1\w{1}\s*$']
    if any(re.match(pattern, token) for pattern in rules_1_2):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rules_3_4):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_5_6):
        return 'NOUN'
    elif re.match(rule_7, token):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_8_9):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rule_10):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rule_11):
        return 'VERB'
    else:
        return 'NOUN'
    
tags = [tag(token) for token in lemmatized_tokens]
print("Lemmatized token list: ", lemmatized_tokens)
print("Custom POS Tagged: ", tags)

Lemmatized token list:  ['زالت', 'تصريح', 'عبد', 'لطيف', 'جواهر', 'والى', 'بن', 'مغرب', 'عقب', 'اخر', 'اجتماع', 'مجلس', 'الاداري', 'لبن', 'مركز', 'موضوع', 'تحليل', 'اوساط', 'خبراء', 'اقتصادي', 'مغرب', 'خارج', 'جزم', 'بش', 'المحافظه', 'معدل', 'الفاءده', 'الرءيسي', 'حال', 'الماءه', 'عدم', 'خلال', 'اجتماع', 'مقبل', 'مجلس', 'حلول', 'مقبل', 'مكتف', 'بالتاكيد', 'قرار', 'كون', 'رهين', 'بالمءشرات', 'المسجله', 'خلال', 'فصول', 'لسن', 'الجاريه', 'ترك', 'مجال', 'مفتوح', 'جميع', 'توقع', 'الحكومه', 'مستعد', 'مواجه', 'موجه', 'تضخم', 'جديد', 'رمض', 'سءال', 'فتء', 'خبراء', 'اقتصادي', 'طرح', 'ظل', 'غموض', 'توقع', 'المرتبطه', 'تضخم', 'توجه', 'بن', 'مغرب', 'خصوص', 'خلال', 'فترة', 'الماضيه', 'والحاليه', 'تعيش', 'المملكه', 'فتر', 'ركود', 'تضخم', 'ارتفاع', 'اسعار', 'خضر', 'بعض', 'مواد', 'الاستهلاكيه', 'شكل', 'تدريج', 'نصف', 'ثاني', 'شهر', 'فضيل', 'تجدد', 'حديث', 'رفع', 'دعم', 'غاز', 'بوتان', 'عزز', 'توقع', 'بش', 'موجه', 'غلاء', 'مرتقب', 'بذل', 'الحكومه', 'نهايه', 'لسن', 'الماضيه', 'مجهود', 'مهم', 'ضمان', 'تو

### Named Entity Recognition

In [65]:
ner = FarasaNamedEntityRecognizer()
ner_tags = ner.recognize(text)
print("NER Tagged text: ",ner_tags)

NER Tagged text:  ما/O زالت/O تصريحات/O عبد/B-PERS اللطيف/I-PERS الجواهري/I-PERS ،/O والي/O بنك/O المغرب/B-LOC ،/O عقب/O آخر/O اجتماع/O للمجلس/O الإداري/O للبنك/B-ORG المركزي/I-ORG ،/O موضوع/O تحليل/O بين/O أوساط/O الخبراء/O الاقتصاديين/O بالمغرب/B-LOC والخارج/O ،/O حيث/O لم/O يجزم/O بشأن/O المحافظة/O على/O معدل/O الفائدة/O الرئيسي/O الحالي/O عند/O 3/O في/O المائة/O من/O عدمه/O ،/O خلال/O الاجتماع/O المقبل/O للمجلس/O بحلول/O يونيو/O المقبل/O ،/O مكتفيا/O بالتأكيد/O على/O أن/O القرار/O سيكون/O رهينا/O بالمؤشرات/O المسجلة/O خلال/O ثاني/O فصول/O السنة/O الجارية/O ؛/O ما/O يترك/O المجال/O مفتوحا/O لجميع/O التوقعات/O ./O هل/O الحكومة/O مستعدة/O لمواجهة/O موجة/O تضخمية/O جديدة/O بعد/O رمضان/O ؟/O ./O سؤال/O ما/O فتئ/O الخبراء/O الاقتصاديون/O يطرحونه/O في/O ظل/O غموض/O التوقعات/O المرتبطة/O بالتضخم/O وتوجه/O بنك/O المغرب/B-LOC ،/O خصوصا/O خلال/O الفترتين/O الماضية/O والحالية/O ،/O حيث/O تعيش/O المملكة/O فترة/O ركود/O تضخمي/O ؛/O إلا/O أن/O ارتفاع/O أسعار/O الخضر/O وبعض/O المواد/O الاستهلاكية/

## References
- Zerrouki, T., (2023). PyArabic: A Python package for Arabic text. Journal of Open Source Software, 8(84), 4886, https://doi.org/10.21105/joss.04886
- Alkhatib, R. M., Zerrouki, T., Shquier, M. M. A., & Balla, A. (2023). Tashaphyne0.4: A new arabic light stemmer based on rhyzome modeling approach. Information Retrieval Journa, 26(14). doi: https://doi.org/10.1007/s10791-023-09429-y
- T. Zerrouki, Qalsadi, Arabic mophological analyzer Library for python.,  https://pypi.python.org/pypi/qalsadi/
- MagedSaeed. (n.d.). GitHub - MagedSaeed/farasapy: A Python implementation of Farasa toolkit. GitHub. https://github.com/MagedSaeed/farasapy?tab=readme-ov-file#want-to-cite
- Hegazi, M. O., Al-Dossari, Y., Al-Yahy, A., Al-Sumari, A., & Hilal, A. (2021). Preprocessing Arabic text on social media. Heliyon, 7(2), e06191. https://doi.org/10.1016/j.heliyon.2021.e06191 
- Sawalha, M., Atwell, E., & Abushariah, M. A. M. (2013). SALMA: Standard Arabic Language Morphological Analysis. 2013 1st International Conference on Communications, Signal Processing, and Their Applications (ICCSPA). https://doi.org/10.1109/iccspa.2013.6487311
- Hjouj, M., Alarabeyyat, A., & Olab, I. (2016). Rule based approach for Arabic part of speech tagging and name entity recognition. International Journal of Advanced Computer Science and Applications, 7(6). https://doi.org/10.14569/ijacsa.2016.070642